In [1]:
import numpy as np
import netCDF4 as nc
import scipy.io as sio

In [2]:
file=sio.loadmat('/global/cscratch1/sd/ashesh/MyQuota/ERA5/Data_assimilation/ERA_grid.mat')
lat=file['lat']
lon=file['lon']

In [3]:
pwd

'/global/cscratch1/sd/ashesh/MyQuota/ERA5/Data_assimilation/Unet_experiment'

This part of the code takes the hourly Z500 data and computes the mean and standard deviation for normalization 

It takes each netCDF file as one entry to a file list to facilitate memory optimization 

In this [paper](https://gmd.copernicus.org/preprints/gmd-2021-71/) year 1979-2017 is taken together, where 1979-2016 is used as training and 2017 as validation. 

In this example, we have changed the validation set slightly. It would not affect the performance. 



In [4]:
file=nc.Dataset('/global/cscratch1/sd/ashesh/MyQuota/ERA5/ERA_Z500_1hour.nc')
Z=np.asarray(file['input'])
M=np.mean(Z.flatten())
sdev=np.std(Z.flatten())

In [5]:
fileList_train = []
fileList_test=[]
for i in range (1979,2017):
    fileList_train.append ('/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_' + str(i)+'_5.625deg.nc')

fileList_test.append('/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_2018_5.625deg.nc')

In [6]:
import tensorflow
import keras.backend as K
#from data_manager import ClutteredMNIST
#from visualizer import plot_mnist_sample
#from visualizer import print_evaluation
#from visualizer import plot_mnist_grid
import netCDF4
import numpy as np
from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D
import keras
from keras.callbacks import History
history = History()

import keras
from keras.layers import Conv2D, Conv2DTranspose, Cropping2D, Concatenate, ZeroPadding2D, merge
from keras.models import load_model
from keras.layers import Input
from keras.models import Model
from keras.layers import Activation
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Dense

from utils import get_initial_weights
from layers import BilinearInterpolation

__version__ = 0.1 

Using TensorFlow backend.


This section is on the circular convolution implementation. 
There is very little performance change between regular Convolution 2D with 0 padding and CConv2D for this problem. 
Ensure both models (during training and autoregressive prediction used the same conv function)

In [ ]:
def CConv2D(filters, kernel_size, strides=(1, 1), activation='linear', padding='valid', kernel_initializer='glorot_uniform', kernel_regularizer=None):
    def CConv2D_inner(x):
        # padding (see https://www.tensorflow.org/api_guides/python/nn#Convolution)
        in_height = int(x.get_shape()[1])
        in_width = int(x.get_shape()[2])

        if (in_height % strides[0] == 0):
            pad_along_height = max(kernel_size[0] - strides[0], 0)
        else:
            pad_along_height = max(
                kernel_size[0] - (in_height % strides[0]), 0)
        if (in_width % strides[1] == 0):
            pad_along_width = max(kernel_size[1] - strides[1], 0)
        else:
            pad_along_width = max(kernel_size[1] - (in_width % strides[1]), 0)

        pad_top = pad_along_height // 2
        pad_bottom = pad_along_height - pad_top
        pad_left = pad_along_width // 2
        pad_right = pad_along_width - pad_left

        # left and right side for padding
        pad_left = Cropping2D(cropping=((0, 0), (in_width-pad_left, 0)))(x)
        pad_right = Cropping2D(cropping=((0, 0), (0, in_width-pad_right)))(x)

        # add padding to incoming image
        conc = Concatenate(axis=2)([pad_left, x, pad_right])

        # top/bottom padding options
        if padding == 'same':
            conc = ZeroPadding2D(padding={'top_pad': pad_top,
                                          'bottom_pad': pad_bottom})(conc)
        elif padding == 'valid':
            pass
        else:
            raise Exception('Padding "{}" does not exist!'.format(padding))

        # perform the circular convolution
        cconv2d = Conv2D(filters=filters, kernel_size=kernel_size,
                         strides=strides, activation=activation,
                         padding='valid',
                         kernel_initializer=kernel_initializer,
                         kernel_regularizer=kernel_regularizer)(conc)

        # return circular convolution layer
        return cconv2d
    return CConv2D_inner

We have defined the model as "stn" where "stn" includes an U-NET with a spatial transformer module that includes an affine transformation and Biliniear interpolation

In [7]:
def stn(input_shape=(32, 64, 1), sampling_size=(8, 16), num_classes=10):
    inputs = Input(shape=input_shape)
    conv1 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv3)


    conv5 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv3)
    conv5 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv5)
    
    locnet = Flatten()(conv5)
    locnet = Dense(500)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(200)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(100)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(50)(locnet)
    locnet = Activation('relu')(locnet)
    weights = get_initial_weights(50)
    locnet = Dense(6, weights=weights)(locnet)
    x = BilinearInterpolation(sampling_size)([inputs, locnet])


    up6 = keras.layers.Concatenate(axis=-1)([Convolution2D(32, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(x)), conv2])
    conv6 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv6)

    up7 = keras.layers.Concatenate(axis=-1)([Convolution2D(32, 2, 2,activation='relu', border_mode='same')(UpSampling2D(size=(2, 2))(conv6)), conv1])
    conv7 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(32, 5, 5, activation='relu', border_mode='same')(conv7)



    conv10 = Convolution2D(1, 5, 5, activation='linear',border_mode='same')(conv7)

    model = Model(input=inputs, output=conv10)



    return model

In [8]:
model = stn()
model.compile(loss='mse', optimizer='adam')
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  app.launch_new_instance()
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/p

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 64, 32)   832         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 64, 32)   25632       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 16, 32, 32)   0           conv2d_2[0][0]                   
____________________________________________________________________________________________

/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:48: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/i

Here, batch_size, and num_epochs have undergone HPO. Change variable "lead" for different "x" in U-STNx. Refer to [paper](https://gmd.copernicus.org/preprints/gmd-2021-71/) for details. For each value of lead a different weights file (including biases) would be saved. 

In [9]:
batch_size = 10
num_epochs = 8
lead=1
count=0
for loop in fileList_train:
    print('******************** counter*************',count)
    File=nc.Dataset(loop)
    Z=np.asarray(File['z'])
    trainN=np.size(Z,0)-300
    Z=(Z-M)/sdev
    
    x_train=Z[0:trainN,:,:]
    x_train=x_train.reshape([np.size(x_train,0),32,64,1])
    y_train=Z[lead:trainN+lead,:,:]
    y_train=y_train.reshape([np.size(y_train,0),32,64,1])
    
    x_val= Z[trainN+lead:np.size(Z,0)-lead,:,:]
    x_val=x_val.reshape([np.size(x_val,0),32,64,1])
    
    y_val= Z[trainN+lead*2:np.size(Z,0),:,:]
    y_val=y_val.reshape([np.size(y_val,0),32,64,1])

    if (count>0):
        
        model = stn()
        model.compile(loss='mse', optimizer='adam')
        model.load_weights('best_weights_lead1.h5')
        hist = model.fit(x_train, y_train,
                       batch_size = batch_size,
             verbose=1,
             epochs = 20,
             validation_data=(x_val,y_val),shuffle=True,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # just to make sure we use a lot of patience before stopping
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('best_weights_lead'+str(lead)+'.h5', monitor='val_loss',
                                                    verbose=1, save_best_only=True,
                                                    save_weights_only=True, mode='auto', period=1),history]
             )

    else:
        hist = model.fit(x_train, y_train,
                       batch_size = batch_size,
             verbose=1,
             epochs = 20,
             validation_data=(x_val,y_val),shuffle=True,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # just to make sure we use a lot of patience before stopping
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('best_weights_lead'+str(lead)+'.h5', monitor='val_loss',
                                                    verbose=1, save_best_only=True,
                                                    save_weights_only=True, mode='auto', period=1),history]
             )


    count=count+1

******************** counter************* 0

Train on 8460 samples, validate on 276 samples
Epoch 1/20
8460/8460 [==============================] - 12s 1ms/step - loss: 0.0263 - val_loss: 0.0085

Epoch 00001: val_loss improved from inf to 0.00845, saving model to best_weights_lead1.h5
Epoch 2/20
8460/8460 [==============================] - 8s 957us/step - loss: 0.0065 - val_loss: 0.0069

Epoch 00002: val_loss improved from 0.00845 to 0.00695, saving model to best_weights_lead1.h5
Epoch 3/20
8460/8460 [==============================] - 8s 949us/step - loss: 0.0054 - val_loss: 0.0082

Epoch 00003: val_loss did not improve from 0.00695
Epoch 4/20
8460/8460 [==============================] - 8s 952us/step - loss: 0.0048 - val_loss: 0.0061

Epoch 00004: val_loss improved from 0.00695 to 0.00613, saving model to best_weights_lead1.h5
Epoch 5/20
8460/8460 [==============================] - 8s 963us/step - loss: 0.0043 - val_loss: 0.0060

Epoch 00005: val_loss improved from 0.00613 to 0.00599,

/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  app.launch_new_instance()
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/p

Train on 8484 samples, validate on 276 samples
Epoch 1/20
8484/8484 [==============================] - 9s 1ms/step - loss: 0.0038 - val_loss: 0.0044

Epoch 00001: val_loss improved from inf to 0.00439, saving model to best_weights_lead1.h5
Epoch 2/20
8484/8484 [==============================] - 8s 924us/step - loss: 0.0025 - val_loss: 0.0044

Epoch 00002: val_loss did not improve from 0.00439
Epoch 3/20
8484/8484 [==============================] - 8s 926us/step - loss: 0.0022 - val_loss: 0.0046

Epoch 00003: val_loss did not improve from 0.00439
Epoch 4/20
8484/8484 [==============================] - 8s 946us/step - loss: 0.0020 - val_loss: 0.0049

Epoch 00004: val_loss did not improve from 0.00439
Epoch 5/20
8484/8484 [==============================] - 8s 968us/step - loss: 0.0019 - val_loss: 0.0048

Epoch 00005: val_loss did not improve from 0.00439
Epoch 6/20
8484/8484 [==============================] - 8s 969us/step - loss: 0.0018 - val_loss: 0.0048

Epoch 00006: val_loss did not i

/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  app.launch_new_instance()
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/p

Train on 8460 samples, validate on 276 samples
Epoch 1/20
8460/8460 [==============================] - 10s 1ms/step - loss: 0.0033 - val_loss: 0.0040

Epoch 00001: val_loss improved from inf to 0.00398, saving model to best_weights_lead1.h5
Epoch 2/20
8460/8460 [==============================] - 8s 986us/step - loss: 0.0024 - val_loss: 0.0040

Epoch 00002: val_loss did not improve from 0.00398
Epoch 3/20
8460/8460 [==============================] - 8s 995us/step - loss: 0.0021 - val_loss: 0.0042

Epoch 00003: val_loss did not improve from 0.00398
Epoch 4/20
8460/8460 [==============================] - 8s 989us/step - loss: 0.0020 - val_loss: 0.0041

Epoch 00004: val_loss did not improve from 0.00398
Epoch 5/20
8460/8460 [==============================] - 8s 987us/step - loss: 0.0019 - val_loss: 0.0041

Epoch 00005: val_loss did not improve from 0.00398
Epoch 6/20
8460/8460 [==============================] - 8s 984us/step - loss: 0.0017 - val_loss: 0.0041

Epoch 00006: val_loss did not 

...

Train on 8460 samples, validate on 276 samples
Epoch 1/20
8460/8460 [==============================] - 19s 2ms/step - loss: 0.0025 - val_loss: 0.0030

Epoch 00001: val_loss improved from inf to 0.00301, saving model to best_weights_lead1.h5
Epoch 2/20
8460/8460 [==============================] - 10s 1ms/step - loss: 0.0019 - val_loss: 0.0026

Epoch 00002: val_loss improved from 0.00301 to 0.00265, saving model to best_weights_lead1.h5
Epoch 3/20
8460/8460 [==============================] - 10s 1ms/step - loss: 0.0017 - val_loss: 0.0028

Epoch 00003: val_loss did not improve from 0.00265
Epoch 4/20
8460/8460 [==============================] - 10s 1ms/step - loss: 0.0016 - val_loss: 0.0028

Epoch 00004: val_loss did not improve from 0.00265
Epoch 5/20
8460/8460 [==============================] - 10s 1ms/step - loss: 0.0015 - val_loss: 0.0032

Epoch 00005: val_loss did not improve from 0.00265
Epoch 6/20
8460/8460 [==============================] - 10s 1ms/step - loss: 0.0014 - val_loss: 

/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  app.launch_new_instance()
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/p

Train on 8460 samples, validate on 276 samples
Epoch 1/20
8460/8460 [==============================] - 19s 2ms/step - loss: 0.0026 - val_loss: 0.0042

Epoch 00001: val_loss improved from inf to 0.00423, saving model to best_weights_lead1.h5
Epoch 2/20
8460/8460 [==============================] - 11s 1ms/step - loss: 0.0020 - val_loss: 0.0045

Epoch 00002: val_loss did not improve from 0.00423
Epoch 3/20
8460/8460 [==============================] - 11s 1ms/step - loss: 0.0018 - val_loss: 0.0046

Epoch 00003: val_loss did not improve from 0.00423
Epoch 4/20
8460/8460 [==============================] - 11s 1ms/step - loss: 0.0016 - val_loss: 0.0049

Epoch 00004: val_loss did not improve from 0.00423
Epoch 5/20
8460/8460 [==============================] - 11s 1ms/step - loss: 0.0016 - val_loss: 0.0044

Epoch 00005: val_loss did not improve from 0.00423
Epoch 6/20
8460/8460 [==============================] - 11s 1ms/step - loss: 0.0015 - val_loss: 0.0045

Epoch 00006: val_loss did not impro

/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
  app.launch_new_instance()
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/python3.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same")`
/usr/common/software/tensorflow/gpu-tensorflow/1.15.0-py37/lib/p

Train on 8484 samples, validate on 276 samples
Epoch 1/20
8484/8484 [==============================] - 20s 2ms/step - loss: 0.0027 - val_loss: 0.0031

Epoch 00001: val_loss improved from inf to 0.00307, saving model to best_weights_lead1.h5
Epoch 2/20
8484/8484 [==============================] - 11s 1ms/step - loss: 0.0020 - val_loss: 0.0032

Epoch 00002: val_loss did not improve from 0.00307
Epoch 3/20
8484/8484 [==============================] - 11s 1ms/step - loss: 0.0017 - val_loss: 0.0032

Epoch 00003: val_loss did not improve from 0.00307
Epoch 4/20
8484/8484 [==============================] - 11s 1ms/step - loss: 0.0016 - val_loss: 0.0034

Epoch 00004: val_loss did not improve from 0.00307
Epoch 5/20
8484/8484 [==============================] - 11s 1ms/step - loss: 0.0015 - val_loss: 0.0034

Epoch 00005: val_loss did not improve from 0.00307
Epoch 6/20
8484/8484 [==============================] - 11s 1ms/step - loss: 0.0014 - val_loss: 0.0033

Epoch 00006: val_loss did not impro

In [10]:
fileList_train

['/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1979_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1980_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1981_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1982_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1983_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1984_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1985_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1986_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1987_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1988_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1989_5.625deg.nc',
 '/global/cscratch1/sd/ashesh/MyQuota/ERA5/geopotential_500hPa_1990_5.625deg.nc',
 '/global/cscrat